# Fine-tuninig the LLM Model
Mahan Madani - Mohammad Mehdi Begmaz

## Load Dataset and important libraries

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW, get_scheduler

import pandas as pd
import numpy as np
import torch.nn as nn

# from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
from torch.utils.data import DataLoader, Dataset

c:\Users\Mahan\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\trainer_pt_utils.py:218: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:77.)
  device: Optional[torch.device] = torch.device("cuda"),


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
df = pd.read_csv("./dataset/BG3_reviews_preprocessed.csv")  # load the preprocessed version of the dataset
print(df.columns)
print(df.shape)

Index(['review', 'voted_up', 'votes_up', 'votes_funny', 'weighted_vote_score',
       'word_count', 'profanity'],
      dtype='object')
(10000, 7)


## Model

In [3]:
model_name = 'gpt2'
model = AutoModelForCausalLM.from_pretrained(model_name)

In [4]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

## Tokenization

In [5]:
class CustomDataset(Dataset):
    def __init__(self, text_list, tokenizer, max_length=512):
        self.input_ids = tokenizer(text_list, truncation=True, max_length=max_length, return_tensors="pt", padding=True)

    def __len__(self):
        return len(self.input_ids["input_ids"])

    def __getitem__(self, idx):
        return {"input_ids": self.input_ids["input_ids"][idx]}

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token 
batch_size = 128

train_dataset = CustomDataset(df['review'].to_list(), tokenizer)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# define an evaluation function to pass into trainer later
def compute_metrics(p):
    loss = p.loss
    return {loss}

# Fine-tune model

In [12]:
def print_trainable_parameters(model):

    # Prints the number of trainable parameters in the model.

    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [13]:
print_trainable_parameters(model)

trainable params: 124439808 || all params: 124439808 || trainable%: 100.0


In [14]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 589824 || all params: 125029632 || trainable%: 0.4717473694555863


c:\Users\Mahan\AppData\Local\Programs\Python\Python310\lib\site-packages\peft\tuners\lora\model.py:347: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [15]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10

In [16]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_name + "-lora-review_generation",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [16]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=tokenized_dataset["train"],
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

NameError: name 'tokenized_dataset' is not defined

In [ ]:
# save model parameters
model.save_pretrained("your_fine_tuned_model_directory")
tokenizer.save_pretrained("your_fine_tuned_model_directory")

# Generate Reviews

In [ ]:
generated_text = model.generate(max_length=100)
print()